In [407]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import datetime

## 코로나 지역별

In [418]:
key_fd = open('./data/corona_key.txt')
key = key_fd.read(200)
key_fd.close()

In [419]:
month_11 = pd.date_range(start='2020-01-01',end='2020-11-30',freq='d')
# 빈 데이터프레임 생성
corona_area = pd.DataFrame(
    columns = ['지역','등록시간','확진자 수','격리해제 수','사망자 수']
    # index = month_11
)
corona_area.head()

,지역,등록시간,확진자 수,격리해제 수,사망자 수


In [424]:
gubun = []
createDt = []
defCnt = []
deathCnt = []
clearCnt = []
for i in tqdm_notebook(range(len(month_11)-1)):
    corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
    startDate = month_11[i].strftime('%Y%m%d')
    endDate = month_11[i].strftime('%Y%m%d')
    url = f'{corona_url}?serviceKey={key}&pageNo=1&numOfRows=10&startCreateDt={startDate}&endCreateDt={endDate}' 
    result = requests.get(urlparse(url).geturl())
    html = result.text
    soup = BeautifulSoup(html, 'html.parser')
    print(soup)
    for i in range(len(soup.select('gubun'))):
        if soup.select('gubun')[i].string in ['검역','합계']:
            # print(soup.select('gubun')[i].string)
            continue
        gubun.append(soup.select('gubun')[i].get_text())
        createDt.append(soup.select('createDt')[i].get_text())
        defCnt.append(soup.select('defCnt')[i].get_text() if not len(soup.select('defCnt'))!= 0 else '')
        clearCnt.append(soup.select('isolClearCnt')[i].get_text() if not len(soup.select('isolClearCnt'))!= 0 else '')
        deathCnt.append(soup.select('deathCnt')[i].get_text())
corona_area['등록시간']=[i for i in createDt]
corona_area['지역'] = gubun
corona_area['확진자 수'] = defCnt
corona_area['격리해제 수'] = clearCnt
corona_area['사망자 수'] = deathCnt


<resultcode>99</resultcode><resultmsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultmsg></header></response>
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultcode>99</resultcode><resultmsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultmsg></header></response>
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultcode>99</resultcode><resultmsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultmsg></header></response>
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultcode>99</resultcode><resultmsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultmsg></header></response>
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultcode>99</resultcode><resultmsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultmsg></header></response>
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultcode>99</resultcode><resultm

In [422]:
corona_area.head()

,지역,등록시간,확진자 수,격리해제 수,사망자 수


In [411]:
# 숫자형으로 바꿈
corona_area[['확진자 수','격리해제 수','사망자 수']] = corona_area[['확진자 수','격리해제 수','사망자 수']].apply(pd.to_numeric)

In [412]:
# 등록시간에서 날짜만 남김
corona_area['등록시간'] = pd.to_datetime(corona_area['등록시간']).dt.date
corona_area['등록시간'] = pd.to_datetime(corona_area['등록시간'])
print(corona_area['등록시간'])
# 피봇으로 정리
corona_area_week = pd.pivot_table(data=corona_area,columns='등록시간',index='지역')
corona_area_week

Series([], Name: 등록시간, dtype: datetime64[ns])


지역


In [413]:
corona_area_week.T.reset_index().set_index('등록시간').index 

DatetimeIndex([], dtype='datetime64[ns]', name='등록시간', freq=None)

In [414]:
corona_area_week.T.reset_index().set_index('등록시간').resample('w').sum()

지역,level_0
등록시간,


In [415]:
# monthly_corona = corona_area.set_index('등록시간')
# monthly_corona = monthly_corona.resample('MS')
# monthly_corona['등록시간'] = corona_area['등록시간'].strftime('%m월')
monthly_corona

,지역,등록시간,확진자 수,격리해제 수,사망자 수
0,제주,2020-11-01 09:42:56.006,59,59,0
1,경남,2020-11-01 09:42:56.006,318,299,0
2,경북,2020-11-01 09:42:56.006,1587,1516,57
3,전남,2020-11-01 09:42:56.006,188,174,2
4,전북,2020-11-01 09:42:56.006,166,149,0
...,...,...,...,...,...
522,광주,2020-11-29 09:42:34.125,683,564,3
523,인천,2020-11-29 09:42:34.124,1376,1068,11
524,대구,2020-11-29 09:42:34.124,7228,6998,199
525,부산,2020-11-29 09:42:34.124,757,596,15


In [416]:
for i in range(len(month_11)):
    corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
    startDate = month_11[i].strftime('%Y%m%d')
    endDate = month_11[i].strftime('%Y%m%d')
    url = f'{corona_url}?serviceKey={key}&pageNo=1&numOfRows=10&startCreateDt={startDate}&endCreateDt={endDate}' 
    result = requests.get(urlparse(url).geturl())
    html = result.text
    soup = BeautifulSoup(html, 'html.parser')
    createDt.append(soup.select('createDt')[0].string)
   

IndexError: list index out of range

In [152]:
soup.select('deathcnt')[0].string

'526'

In [115]:
pd.date_range(start='2020-11-01',end='2020-11-30',freq='d')

Timestamp('2020-11-01 00:00:00', freq='D')

### 코로나현황

In [161]:
month_11 = pd.date_range(start='2020-11-01',end='2020-11-30',freq='d')
corona = pd.DataFrame(
    columns = ['등록시간','게시글번호','확진자 수','격리해제 수','검사진행 수','사망자 수','치료중 환자 수']
    # index = month_11
)
corona.head()

,등록시간,게시글번호,확진자 수,격리해제 수,검사진행 수,사망자 수,치료중 환자 수


In [164]:
createDt = []
seq = []
decideCnt = []
clearCnt = []
accExamCnt = []
deathCnt = []
careCnt = []
for i in range(len(month_11)):
    corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
    startDate = month_11[i].strftime('%Y%m%d')
    endDate = month_11[i].strftime('%Y%m%d')
    url = f'{corona_url}?serviceKey={key}&pageNo=1&numOfRows=10&startCreateDt={startDate}&endCreateDt={endDate}' 
    result = requests.get(urlparse(url).geturl())
    html = result.text
    soup = BeautifulSoup(html, 'html.parser')
    createDt.append(soup.select('createDt')[0].string)
    seq.append(soup.select('seq')[0].string)
    decideCnt.append(soup.select('decideCnt')[0].string)
    clearCnt.append(soup.select('clearCnt')[0].string)
    accExamCnt.append(soup.select('accExamCnt')[0].string)
    deathCnt.append(soup.select('deathCnt')[0].string)
    careCnt.append(soup.select('careCnt')[0].string)
corona['등록시간']=[str(i) for i in createDt]
corona['게시글번호']=seq
corona['확진자 수']=decideCnt
corona['격리해제 수']=clearCnt
corona['검사진행 수']=accExamCnt
corona['사망자 수']=deathCnt
corona['치료중 환자 수']=careCnt
corona

,등록시간,게시글번호,확진자 수,격리해제 수,검사진행 수,사망자 수,치료중 환자 수
0,2020-11-01 09:41:40.025,309,26635,24357,2630630,466,1812
1,2020-11-02 09:35:09.923,310,26732,24395,2636650,468,1869
2,2020-11-03 09:30:37.135,311,26807,24510,2649859,472,1825
3,2020-11-04 09:37:25.13,312,26925,24616,2662260,474,1835
4,2020-11-05 09:51:34.898,313,27050,24735,2673706,475,1840
5,2020-11-06 09:24:23.717,314,27195,24821,2686314,476,1898
6,2020-11-07 09:40:32.259,315,27284,24910,2697249,477,1897
7,2020-11-08 09:34:44.609,316,27427,24968,2702880,478,1981
8,2020-11-09 09:36:39.787,317,27553,25029,2709199,480,2044
9,2020-11-10 09:37:02.955,318,27653,25160,2723960,485,2008
